In [1]:
# %load_ext autoreload
# %autoreload 2

# %pip install -r requirements.txt

In [2]:
import sys
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
import torchvision.models as models
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets
import pickle

In [3]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Check for CUDA support
print(f"Is CUDA available? {torch.cuda.is_available()}")

# Set the device
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using device: {device}")


PyTorch version: 2.1.0+cu118
Is MPS (Metal Performance Shader) built? False
Is MPS available? False
Is CUDA available? True
Using device: cuda


In [4]:
model_type = "densenet_cifar100" # either "resnet" or "vgg_cifar10" or "vgg_cifar100"

model_path = os.path.join("models", model_type)

num_classes = 100 if "cifar100" in model_type else 10

In [5]:
from DataLoader import CustomDataset
import numpy as np

if model_type == "resnet":
    # use the imagenette dataset
    hf_dataset = load_dataset("frgfm/imagenette", '320px')
    hf_dataset = concatenate_datasets(hf_dataset.values())
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

elif model_type == "vgg_cifar10":
    # use the cifar10 dataset
    hf_dataset = load_dataset("cifar10")
    hf_dataset = concatenate_datasets(hf_dataset.values())
    
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.507, 0.4865, 0.4409],
                             std=[0.2673, 0.2564, 0.2761])
    ])
elif model_type == "vgg_cifar100" or model_type == "densenet_cifar100":
    # use the cifar100 dataset
    hf_dataset = load_dataset("cifar100")
    hf_dataset = concatenate_datasets(hf_dataset.values())
    
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
    ])
    
torch_dataset = CustomDataset(hf_dataset, transform=transform)

batch_size = 32 if model_type == "resnet" else 64

test_size = 0.2
test_volume = int(test_size * len(torch_dataset))
train_volume = len(torch_dataset) - test_volume

train_dataset, test_dataset = random_split(torch_dataset, [train_volume, test_volume])
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=False, 
    num_workers=4
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4
)

In [6]:
import ModelLoader

loader = ModelLoader.ModelLoader(model_type, device)

model = loader.load_model(num_outputs=num_classes)

model

Loading EarlyExit DenseNet121 model architecture...
Adding exits...


EarlyExitModel(
  (model): DenseNet(
    (conv1): Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (dense1): Sequential(
      (0): Bottleneck(
        (bn1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(24, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (1): Bottleneck(
        (bn1): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(36, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (2): Bottleneck(
        (bn1): BatchNorm2d(48, eps

## Early Exit Model Training

If you want tensorboard support, you need to run the following commands on **macOS**:

This clears your logs:
```
rm -rf runs/*
```

This shows your public IP address:
```
dig -4 TXT +short o-o.myaddr.l.google.com @ns1.google.com
```

This starts tensorboard at this address:
```
tensorboard --host 0.0.0.0 --logdir={model_path}/runs
```
```



If you want tensorboard support, you need to run the following commands on **Windows**:

This clears your logs:
```
rmdir /s /q runs
```

This shows your IP address:
```
powershell -command "$ipAddress = (Invoke-WebRequest -Uri 'http://ipinfo.io/ip').Content.Trim(); Write-Host 'Your IP address is: ' $ipAddress"  
```

This starts tensorboard at this address:
```
tensorboard --host 0.0.0.0 --logdir={model_path}\runs
```

In [7]:
from EarlyExitTrainer import ModelTrainer

trainer = ModelTrainer(model, device, model_dir=model_path)

In [8]:
# train the classifiers
trainer.train_classifiers(train_dataloader, epoch_count=30, validation_loader=test_dataloader)


Training classifier for exit 1


Epoch 0 Loss 3.626804178873698
Epoch 0 Accuracy 0.22164583333333332
Epoch 0 Validation Loss 3.1221434448627714
Epoch 0 Validation Accuracy 0.29596077127659576


Epoch 1 Loss 2.8041664861043296
Epoch 1 Accuracy 0.3617708333333333
Epoch 1 Validation Loss 2.858804517603935
Epoch 1 Validation Accuracy 0.3346908244680851


Epoch 2 Loss 2.537349071184794
Epoch 2 Accuracy 0.41222916666666665
Epoch 2 Validation Loss 2.7860132151461663
Epoch 2 Validation Accuracy 0.34507978723404253


Epoch 3 Loss 2.3903164149920144
Epoch 3 Accuracy 0.4448125
Epoch 3 Validation Loss 2.7754577550482242
Epoch 3 Validation Accuracy 0.3507313829787234


...

Epoch 25 Loss 0.5779502451817194
Epoch 25 Accuracy 0.8648333333333333
Epoch 25 Validation Loss 0.5317449408801312
Epoch 25 Validation Accuracy 0.8797373670212766


Epoch 26 Loss 0.5825937119026979
Epoch 26 Accuracy 0.8648333333333333
Epoch 26 Validation Loss 0.5363723434349323
Epoch 26 Validation Accuracy 0.8797373670212766


Epoch 27 Loss 0.5873580257892609
Epoch 27 Accuracy 0.8647916666666666
Epoch 27 Validation Loss 0.5410728501354126
Epoch 27 Validation Accuracy 0.8794880319148937


Epoch 28 Loss 0.5922221521536509
Epoch 28 Accuracy 0.8646458333333333
Epoch 28 Validation Loss 0.5458329455491077
Epoch 28 Validation Accuracy 0.8794880319148937


Epoch 29 Loss 0.5971557800869147
Epoch 29 Accuracy 0.8648541666666667
Epoch 29 Validation Loss 0.5506236114876067
Epoch 29 Validation Accuracy 0.8795711436170213
